In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import sklearn
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
import xgboost as xgb
from xgboost import XGBRegressor

/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [4]:
frac = 1

train = train.sample(frac=frac, random_state=42)

train = train.sort_values(by=['year', 'week'])

X = train.drop(['Num','y'], axis=1)
y = train['y']

In [5]:
def smape(y, y_pred, **kwargs):
    y = np.array(y)
    y_pred = np.array(y_pred)
    ans = (np.abs(y - y_pred) * 1.0 / (np.abs(y) + np.abs(y_pred))).mean() * 100
    return ans * 2

from sklearn.metrics import make_scorer
scorer = make_scorer(score_func=smape)

In [6]:
XX = X.copy()
XX['item'] = XX['item_id']
XX = XX.drop('item_id', axis=1)

Xtest = test.copy()
Xtest['item'] = test['item_id']
Xtest = Xtest.drop('item_id', axis=1)

model = XGBRegressor(n_estimators=100, max_depth=14, learning_rate=0.03, seed=95)
model.fit(XX, y)

preds = model.predict(Xtest.drop(['Num'], axis=1))

In [7]:
sample_submission['y'] = preds

In [8]:
# In GBM you can get some negative predictions:
print(sample_submission[sample_submission['y'] < 0])

         Num         y
1353  350866 -8.096147


In [9]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [10]:
sample_submission.to_csv("sample_submission.tsv", sep=',', index=False)